# Retail Dataset EDA

## Pre-processing

In [26]:
#import library
import pandas as pd
import numpy as np
import calendar

In [49]:
# Import the dataset
df = pd.read_csv('C:/Users/nownu/OneDrive/Desktop/Arfan/project-portfolio/retail/data/Fashion_Retail_Sales.csv')

In [29]:
# Checking the Dtype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer Reference ID  3400 non-null   int64  
 1   Item Purchased         3400 non-null   object 
 2   Purchase Amount (USD)  2750 non-null   float64
 3   Date Purchase          3400 non-null   object 
 4   Review Rating          3076 non-null   float64
 5   Payment Method         3400 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 159.5+ KB


The Dtype of ```Date Purchase``` is object and ```Customer Reference ID``` is Integer, we will change it later in cleaning section


In [43]:
# Checking the data and finding insight
df

,Customer Reference ID,Item Purchased,Purchase Amount (USD),Date Purchase,Review Rating,Payment Method,month
0,4018,Handbag,4619.000000,2023-02-05,2.999057,Credit Card,2
1,4115,Tunic,2456.000000,2023-07-11,2.000000,Credit Card,7
2,4019,Tank Top,2102.000000,2023-03-23,4.100000,Cash,3
3,4097,Leggings,3126.000000,2023-03-15,3.200000,Cash,3
4,3997,Wallet,3003.000000,2022-11-27,4.700000,Cash,11
...,...,...,...,...,...,...,...
3395,4118,Shorts,37.000000,2023-07-09,2.700000,Cash,7
3396,4056,Slippers,176.000000,2023-03-16,2.500000,Cash,3
3397,3991,Onesie,156.709818,2022-10-03,2.400000,Credit Card,10
3398,4042,Sandals,128.000000,2023-01-01,1.800000,Cash,1


We found many question in this data such as:
- What is the most sold products?
- What is the most expensive products?
- What is the number of selling per month?
- What is the high rated products?
- Who has the most total spent?

## Cleaning

In [32]:
# Checking the missing values and duplicated data
df.isna().sum()

Customer Reference ID      0
Item Purchased             0
Purchase Amount (USD)    650
Date Purchase              0
Review Rating            324
Payment Method             0
dtype: int64

In [46]:
df.duplicated().sum()

np.int64(0)

As you can see, we dont got much problem in here. There is no duplicated data and only missing values which we can fill it with mean or the average values of it, thank god I got this dataset.

In [48]:
# Changing the dtype of Date purchase
df['Date Purchase'] = pd.to_datetime(
    df['Date Purchase'], dayfirst=True, errors='coerce')

In [47]:
# Filling the missing values with the average values
df['Purchase Amount (USD)'] = df['Purchase Amount (USD)'].fillna(
    df['Purchase Amount (USD)'].mean())
df['Review Rating'] = df['Review Rating'].fillna(
    df['Review Rating'].mean())

## Processing Data

In [37]:
# Finfing Top 5 most sold products
produk_terlaris = df.groupby('Item Purchased')['Item Purchased'].count().sort_values(ascending=False).reset_index(name='total terjual')
produk_terlaris.head()

,Item Purchased,total terjual
0,Belt,90
1,Skirt,88
2,Shorts,87
3,Pants,86
4,T-shirt,82


In [44]:
# Finding Top 5 most expensive products
produk_termahal = df.groupby('Item Purchased')['Purchase Amount (USD)'].mean().sort_values(ascending=False).reset_index(name='Harga rata-rata (USD)')
produk_termahal['Harga rata-rata (USD)'] = np.ceil(produk_termahal['Harga rata-rata (USD)']).astype(int)
produk_termahal.head()

,Item Purchased,Harga rata-rata (USD)
0,Tunic,302
1,Flip-Flops,253
2,Jeans,253
3,Gloves,228
4,Slippers,227


In [45]:
# Finding the selling number per month
bulan = {
    1: 'Januari', 2: 'Februari', 3: 'Maret', 4: 'April',
    5: 'Mei', 6: 'Juni', 7: 'Juli', 8: 'Agustus',
    9: 'September', 10: 'Oktober', 11: 'November', 12: 'Desember'
}
df['month'] = df['Date Purchase'].dt.month
penjualan_setahun = df.groupby('month')['month'].count().reset_index(name='Total Penjualan')
penjualan_setahun['month'] = penjualan_setahun['month'].map(bulan)
penjualan_setahun

,month,Total Penjualan
0,Januari,292
1,Februari,251
2,Maret,277
3,April,295
4,Mei,308
5,Juni,262
6,Juli,303
7,Agustus,301
8,September,253
9,Oktober,314


In [40]:
# Finding Top 5 high rated products
high_rate_produk = df.groupby('Item Purchased')['Review Rating'].mean().sort_values(ascending=False).reset_index(name='Total rating rata-rata')
high_rate_produk['Total rating rata-rata'] = np.floor(high_rate_produk['Total rating rata-rata'] * 10) / 10
high_rate_produk.head()

,Item Purchased,Total rating rata-rata
0,Sun Hat,3.4
1,Flip-Flops,3.2
2,Sandals,3.2
3,Jeans,3.2
4,Boots,3.1


In [41]:
# Finding Top 5 customer with most total spent
customer_names = pd.Series({
    4040: 'Customer A',
    4109: 'Customer B',
    4044: 'Customer C',
    4108: 'Customer D',
    4075: 'Customer E'
})

rich_man = df.groupby('Customer Reference ID')['Purchase Amount (USD)'].sum().sort_values(ascending=False).reset_index(name='total spent')
rich_man['total spent'] = rich_man['total spent'].round().astype(int)
rich_man['customer_name'] = rich_man['Customer Reference ID'].map(customer_names)
rich_man.head()

,Customer Reference ID,total spent,customer_name
0,4040,10754,Customer A
1,4109,10155,Customer B
2,4044,9685,Customer C
3,4108,7961,Customer D
4,4075,7886,Customer E


## Conclusion

We only do 5 simple analysis that often make people curious, but actually we can anything with this dataset even down to machine learning. We are not gonna do that now, but we will do it soon.

Anyway, the analysis of fashion retail sales data shows fairly stable sales performance throughout the year with the highest peak in October. Products such as Belts, Skirts, and Shorts recorded the highest sales, while Tunics and Jeans were included in the product categories with the highest average prices. In terms of customer satisfaction, products such as Sun Hats and Flip-Flops received the highest review ratings. The analysis also identified five customers with the highest total spending, which contributed significantly to revenue. In addition, the data cleansing process successfully addressed blank values ​​and inconsistent date formats, allowing for more accurate analysis.